In [15]:
#!pip install viola-ml

ERROR: Could not find a version that satisfies the requirement viola-ml (from versions: none)
ERROR: No matching distribution found for viola-ml


ModuleNotFoundError: No module named 'viola_ml'

In [27]:
!pip install viola # Install viola instead of viola-ml

In [19]:
!pip install --force-reinstall viola-ml

ERROR: Could not find a version that satisfies the requirement viola-ml (from versions: none)
ERROR: No matching distribution found for viola-ml


In [28]:
# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')
!pip install mmh3
import pandas as pd
import numpy as np
import re
import ipaddress
from urllib.parse import urlparse
import requests
import mmh3
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 📂 Load dataset from Google Drive
file_path = ""  # Change this to your file location
df = pd.read_csv("/content/drive/MyDrive/urls (1) - test 3.csv")

# 🔍 Ensure correct column names
print("Columns in dataset:", df.columns)
df.columns = df.columns.str.strip()

# 🔹 Select required columns
url_column = "URL"  # Change if needed
label_column = "Flag"  # Change if needed
df = df[[url_column, label_column]]

# 🔎 Feature Extraction Functions
def url_length(url):
    return len(url)

def num_digits(url):
    return sum(c.isdigit() for c in url)

def num_special_chars(url):
    return len(re.findall(r"[@!#$%^&*()]", url))

def num_subdomains(url):
    return urlparse(url).netloc.count('.')

def has_ip(url):
    try:
        ipaddress.ip_address(urlparse(url).netloc)
        return 1
    except ValueError:
        return 0

def is_https(url):
    return 1 if urlparse(url).scheme == "https" else 0

def contains_at_symbol(url):
    return 1 if "@" in url else 0

def contains_email(url):
    return 1 if re.search(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", url) else 0

def double_slash_redirect(url):
    return 1 if "//" in url[7:] else 0

def prefix_suffix(url):
    return 1 if "-" in urlparse(url).netloc else 0

def custom_port(url):
    return 1 if ":" in urlparse(url).netloc else 0

def entropy(url):
    return -sum((url.count(c) / len(url)) * np.log2(url.count(c) / len(url)) for c in set(url)) if url else 0

def favicon_available(url):
    try:
        favicon_url = url.rstrip("/") + "/favicon.ico"
        response = requests.get(favicon_url, timeout=3)
        return 1 if response.status_code == 200 else 0
    except:
        return 0

def suspicious_words(url):
    keywords = ["secure", "account", "login", "banking", "free", "click", "verify", "password"]
    return sum(1 for word in keywords if word in url.lower())

def suspicious_chars(url):
    return 1 if re.search(r"[<>\"'{}]", url) else 0

def sql_injection(url):
    return 1 if re.search(r"(\bSELECT\b|\bUPDATE\b|\bDELETE\b|\bINSERT\b)", url.upper()) else 0

def top_level_domain_encoded(url):
    tld_list = [".com", ".net", ".org", ".info", ".gov", ".edu", ".co", ".biz"]
    return 1 if any(url.endswith(tld) for tld in tld_list) else 0

def query_params(url):
    return len(urlparse(url).query.split("&")) if urlparse(url).query else 0

def path_segments(url):
    return len(urlparse(url).path.split("/")) - 1

# 🔥 Extract Features
df["url_length"] = df[url_column].apply(url_length)
df["num_digits"] = df[url_column].apply(num_digits)
df["num_special_chars"] = df[url_column].apply(num_special_chars)
df["num_subdomains"] = df[url_column].apply(num_subdomains)
df["has_ip"] = df[url_column].apply(has_ip)
df["is_https"] = df[url_column].apply(is_https)
df["contains_at_symbol"] = df[url_column].apply(contains_at_symbol)
df["contains_email"] = df[url_column].apply(contains_email)
df["double_slash_redirect"] = df[url_column].apply(double_slash_redirect)
df["prefix_suffix"] = df[url_column].apply(prefix_suffix)
df["custom_port"] = df[url_column].apply(custom_port)
df["entropy"] = df[url_column].apply(entropy)
df["favicon_available"] = df[url_column].apply(favicon_available)
df["suspicious_words"] = df[url_column].apply(suspicious_words)
df["suspicious_chars"] = df[url_column].apply(suspicious_chars)
df["sql_injection"] = df[url_column].apply(sql_injection)
df["top_level_domain_encoded"] = df[url_column].apply(top_level_domain_encoded)
df["query_params"] = df[url_column].apply(query_params)
df["path_segments"] = df[url_column].apply(path_segments)

# 🎯 Prepare features and labels
feature_columns = df.columns[2:]  # Selecting only extracted features
X = df[feature_columns]
y = df[label_column].values.ravel()

# 🔀 Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 🏆 Train 4 ML Models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=200),
    "SVC": SVC(kernel="linear", C=1),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=200),
}

# 🔍 Evaluate Performance
performance = {}
best_model = None
best_accuracy = 0

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # 📊 Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average="binary", zero_division=1)
    recall = recall_score(y_test, y_pred, average="binary", zero_division=1)
    f1 = f1_score(y_test, y_pred, average="binary")
    auc = roc_auc_score(y_test, y_pred) if len(set(y_test)) > 1 else None

    performance[name] = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "AUC Score": auc,
    }

    # Update best model
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = name

# 📊 Display Results
df_performance = pd.DataFrame(performance).T.round(3)
print("\n📊 Model Performance:")
display(df_performance)

# 🎯 Best Model
print(f"\n🏆 Best Model: {best_model} with {best_accuracy:.3f} Accuracy")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Columns in dataset: Index(['URL', 'Flag'], dtype='object')

📊 Model Performance:


,Accuracy,Precision,Recall,F1 Score,AUC Score
Logistic Regression,0.963,1.000,0.0,0.000,0.500
Random Forest,0.963,0.500,0.2,0.286,0.596
SVC,0.963,1.000,0.0,0.000,0.500
Gradient Boosting,0.956,0.333,0.2,0.250,0.592



🏆 Best Model: Logistic Regression with 0.963 Accuracy


In [29]:
print(df.columns)  # To list column names
print(df.head())   # To see actual data values

Index(['URL', 'Flag', 'url_length', 'num_digits', 'num_special_chars',
       'num_subdomains', 'has_ip', 'is_https', 'contains_at_symbol',
       'contains_email', 'double_slash_redirect', 'prefix_suffix',
       'custom_port', 'entropy', 'favicon_available', 'suspicious_words',
       'suspicious_chars', 'sql_injection', 'top_level_domain_encoded',
       'query_params', 'path_segments'],
      dtype='object')
                                     URL  Flag  url_length  num_digits  \
0  https://www.bluewave.com/akjsbcakjhfb     1          37           0   
1      https://www.redsky.net/qwertyuiop     0          33           0   
2      https://www.greentree.org/zxcvbnm     1          33           0   
3     https://www.bluewave.com/lkjhgfdsa     0          34           0   
4     https://www.silverstone.io/mnbvcxz     1          34           0   

   num_special_chars  num_subdomains  has_ip  is_https  contains_at_symbol  \
0                  0               2       0         1       

In [31]:
!pip install voila colorama tabulate mmh3
import time
import string
import requests
import re
import pandas as pd
import numpy as np
from urllib.parse import urlparse
from sklearn.preprocessing import LabelEncoder
import mmh3
from tabulate import tabulate
from colorama import Fore, Style, init
import ipywidgets as widgets
from IPython.display import display

# Initialize colorama for colored output
init(autoreset=True)

# 🚀 Cool ASCII Header
def print_header():
    display(widgets.HTML(
        f"""<pre style="color: cyan; font-weight: bold;">
   ██████████████████████████████
   █─▄▄▄▄█─▄▄▄─█▄─█─▄█▄─▄▄─█─▄▄▄▄█
   █▄▄▄▄─█─███▀██▄─▄███─▄█▀█▄▄▄▄─█
   ▀▄▄▄▄▄▀▄▄▄▄▄▀▀▄▄▄▀▀▄▄▄▄▄▀▄▄▄▄▄▀
   🔹 URL SAFETY CHECKER 🔹
   ⚡ Powered by Machine Learning & Google Safe Browsing ⚡
        </pre>"""
    ))

# 🌐 Google Safe Browsing Check
def check_google_safebrowsing(url, api_key):
    """Checks the URL using Google's Safe Browsing API."""
    payload = {
        "client": {"clientId": "cyber_scan", "clientVersion": "1.0"},
        "threatInfo": {
            "threatTypes": ["MALWARE", "SOCIAL_ENGINEERING"],
            "platformTypes": ["ANY_PLATFORM"],
            "threatEntryTypes": ["URL"],
            "threatEntries": [{"url": url}]
        }
    }
    response = requests.post(f"https://safebrowsing.googleapis.com/v4/threatMatches:find?key={api_key}", json=payload)
    return "⚠️ Unsafe" if "matches" in response.json() else "✅ Safe"

# 🔹 Feature Extraction Function
def extract_features(url, label_encoder):
    """Extracts multiple features from a given URL."""
    return {
        'url_length': len(url),
        'num_digits': sum(c.isdigit() for c in url),
        'num_special_chars': sum(c in string.punctuation for c in url),
        'num_subdomains': urlparse(url).netloc.count('.'),
        'is_https': 1 if url.lower().startswith("https://") else 0,
        'contains_at_symbol': 1 if "@" in url else 0,
        'contains_email': 1 if re.search(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", url) else 0,
        'entropy': -sum((url.count(c) / len(url)) * np.log2(url.count(c) / len(url)) for c in set(url)) if url else 0,
        'top_level_domain_encoded': label_encoder.transform([urlparse(url).netloc.split('.')[-1]])[0]
    }

# 🔥 Function to Evaluate URLs
def evaluate_urls(url_list, trained_models, df, url_column, label_column, api_key):
    """Evaluates multiple URLs using trained ML models and Google Safe Browsing."""
    results = []

    # 🔹 Ensure Feature Consistency (drop URL & label columns)
    expected_features = df.drop(columns=[url_column, label_column]).columns.tolist()

    # 🔹 Label Encoding for TLD (Top-Level Domain)
    label_encoder = LabelEncoder()
    label_encoder.fit(df[url_column].apply(lambda url: urlparse(url).netloc.split('.')[-1]).unique())

    print_header()
    time.sleep(1)

    for url in url_list:
        if not re.match(r'^(https?|ftp)://[^\s/$.?#].[^\s]*$', url):
            results.append({"URL": url, "Google Safe Browsing": "Invalid URL"})
            continue

        # 🔹 Extract Features & Ensure Feature Consistency
        X_new = pd.DataFrame([extract_features(url, label_encoder)])
        X_new = X_new.reindex(columns=expected_features, fill_value=0)  # Ensure correct column order

        predictions = {name: "⚠️ Phishing" if model.predict(X_new)[0] else "✅ Safe" for name, model in trained_models.items()}
        google_result = check_google_safebrowsing(url, api_key)

        results.append({"URL": url, **predictions, "Google Safe Browsing": google_result})

    # 📊 Display Results in Voila
    results_df = pd.DataFrame(results)
    output_table = widgets.HTML(f"<pre>{tabulate(results_df, headers='keys', tablefmt='html', showindex=False)}</pre>")
    display(output_table)

# 🏆 Create Voila Web Interface
def voila_ui():
    text_area = widgets.Textarea(placeholder="Enter multiple URLs, one per line...", layout=widgets.Layout(width="100%", height="100px"))
    button = widgets.Button(description="Evaluate URLs", button_style="info")

    output = widgets.Output()

    def on_button_click(_):
        with output:
            output.clear_output()
            url_list = text_area.value.split("\n")
            evaluate_urls(url_list, trained_models=models, df=df, url_column="URL", label_column=LABEL_COLUMN, api_key=API_KEY)

    button.on_click(on_button_click)

    display(widgets.VBox([text_area, button, output]))

# 🚀 Run the Tool
API_KEY = ""
LABEL_COLUMN = "Flag"

# Ensure dataset (`df`) and models (`models`) exist before running
if 'df' not in locals() or df is None:
    raise ValueError("❌ Dataset (df) is not loaded. Ensure your dataset is properly initialized.")

if 'models' not in locals() or models is None:
    raise ValueError("❌ Trained ML models (models) are not defined. Ensure your models are properly trained.")

voila_ui()


In [30]:
!voila "voila test of 20+ features.ipynb"


Traceback (most recent call last):
  File "/usr/local/bin/voila", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 991, in launch_instance
    app.initialize(argv)
  File "/usr/local/lib/python3.11/dist-packages/voila/app.py", line 561, in initialize
    raise ValueError(
ValueError: argument is neither a file nor a directory: 'voila test of 20+ features.ipynb'
